# Empirical Model Learning for Black Box Optimization
Master Thesis in Artificial Intelligence at University of Bologna, a.y. 2021/2022

Daniele Verì, Michele Lombardi, Andrea Borghesi, Stefano Teso
___
This notebook is diveded in 4 sections:
- 📥 Dependencies: downloads and install the requirements
- 📑 Definitions: define classes and functions used
- 🚀 Experiments: set parameters, instance targets and run the experiments
- 🧪 Test: test the proper functionality of class and functions

Name convenctions adopted are:
- Functions and local variable are in snake case
- Class are in camel case
- Global variables are in caps lock

📈 You can keep track of the experimets using Weights and Biases.  
**If you don't have an account, disable the corresponding checkbox.**

In order to achieve the true determinism, you have to perform the training on CPU.

# 📥 Dependencies

In [ ]:
#@title GPyopt
!pip install GPyOpt

In [ ]:
#@title CPLEX

!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/69b1b89a73a7d0931fbfdb355eb147c3 -O cplex_studio1210.linux-x86-64.bin
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/97133b747b0114a4e3dba77ab26d68d5 -O response.properties

!pip install docplex
!sh cplex_studio1210.linux-x86-64.bin -f response.properties
!python3 /opt/ibm/ILOG/CPLEX_Studio1210/python/setup.py install

In [ ]:
#@title Tensorflow addons

!pip install tensorflow-addons

In [ ]:
#@title EMLlib

%load_ext autoreload
%autoreload 2

!git clone https://github.com/DanieleVeri/emllib.git

In [ ]:
#@title scikit-optimize
!pip install scikit-optimize

In [ ]:
#@title tensorflow quantization
!pip install -q tensorflow-model-optimization

# 📑 Definitions

In [ ]:
#@title Base import and seed

import os
import math
import random
import numpy as np
import tensorflow as tf

import sys
if not 'emllib' in sys.path: sys.path.insert(1, 'emllib')

import pickle

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

##Problem

In [ ]:
#@title Problem class - quadratic + random slack

import docplex.mp.model as cpx

from skopt.sampler import Lhs
from skopt.space import Space

class ProblemQRS:

    def __init__(self, name, fun, input_type, input_bounds, constraint_cb=None, stocasthic=False):
        self.name = name
        self.fun = fun
        self.input_type = input_type
        self.input_bounds = input_bounds
        self.input_shape = len(self.input_bounds)
        self.constraint_cb = constraint_cb
        self.stocasthic = stocasthic

    def get_dataset(self, n_points):
        if self.constraint_cb is None:
            x = np.random.rand(n_points, self.input_shape)
            for i, b in enumerate(self.input_bounds):
                lb = b[0]
                ub = b[1]
                if self.input_type[i] == "int":
                    x[:,i] = np.random.randint(lb, high=ub, size=n_points)
                else:
                    x[:,i] *= ub - lb
                    x[:,i] += lb
            y = np.zeros((n_points))
            for i in range(n_points):
                y[i] = self.fun(x[i, :])
            return x, y

        # constrained dataset:
        x = np.zeros((n_points, self.input_shape))
        bounds = []
        for i, b in enumerate(self.input_bounds):
            if self.input_type[i] == "int":
                bounds.append((int(b[0]), int(b[1])))
            else:
                bounds.append((float(b[0]), float(b[1])))
                
        space = Space(bounds)
        lhs = Lhs(lhs_type="classic", criterion=None, iterations=1000)
        lhs_samples = lhs.generate(space.dimensions, n_points)

        p = 0
        while True:
            cplex = cpx.Model()
            xvars = []
            for i,b in enumerate(self.input_bounds):
                if self.input_type[i] == "int":
                    xvars.append(cplex.integer_var(lb=b[0], ub=b[1], name="x"+str(i)))
                else:
                    xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))
            
            csts = self.constraint_cb(cplex, xvars)
            # add slack
            avg_range = np.mean(np.diff(np.array(self.input_bounds), axis=1))
            for pc in csts:
                if pc[0].sense.value == 1: # <=
                    if pc[0].right_expr.is_constant() and not pc[0].right_expr.is_zero():
                        pc[0].right_expr -= np.random.uniform()*pc[0].right_expr
                    else:
                        pc[0].right_expr -= np.random.uniform()*avg_range
                elif pc[0].sense.value == 3: # >=
                    if pc[0].right_expr.is_constant() and not pc[0].right_expr.is_zero():
                        pc[0].right_expr += np.random.uniform()*pc[0].right_expr
                    else:
                        pc[0].right_expr += np.random.uniform()*avg_range
                print("\n", pc[0])
                cplex.add_constraint(*pc)

            ## quadratic random objective (boundaries)
            obj = 0
            for i, var in enumerate(xvars):
                obj += (var-lhs_samples[p][i]) ** 2
            cplex.set_objective("min", obj)
            
            # solve
            cplex.set_time_limit(30)
            sol = cplex.solve()
            if sol is None:
                print("infeasible")
                continue
            for i in range(self.input_shape):
                x[p, i] = sol["x"+str(i)]

            p += 1
            if p == n_points: break
        print("Initial points:\n", x)

        # eval fun
        y = np.zeros((n_points))
        for i in range(n_points):
            y[i] = self.fun(x[i, :])

        return x,y

    def get_grid(self, n_points):
        x_list = []
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x_list.append(np.arange(lb, ub, max(1, (ub-lb)//n_points)))
            else:
                x_list.append(np.arange(lb, ub, (ub-lb)/n_points))
        x = np.array(np.meshgrid(*x_list)).reshape(self.input_shape,-1).T
        y = np.zeros((x.shape[0]))
        for i in range(x.shape[0]):
            y[i] = self.fun(x[i, :])
        return x, y

In [ ]:
#@title Problem class - convex interpolation

import docplex.mp.model as cpx
import pandas as pd

class Problem:

    def __init__(self, name, fun, input_type, input_bounds, constraint_cb=None, stocasthic=False):
        self.name = name
        self.fun = fun
        self.input_type = input_type
        self.input_bounds = input_bounds
        self.input_shape = len(self.input_bounds)
        self.constraint_cb = constraint_cb
        self.stocasthic = stocasthic

    def get_dataset(self, n_points):
        if self.constraint_cb is None:
            x = np.random.rand(n_points, self.input_shape)
            for i, b in enumerate(self.input_bounds):
                lb = b[0]
                ub = b[1]
                if self.input_type[i] == "int":
                    x[:,i] = np.random.randint(lb, high=ub, size=n_points)
                else:
                    x[:,i] *= ub - lb
                    x[:,i] += lb
            y = np.zeros((n_points))
            for i in range(n_points):
                y[i] = self.fun(x[i, :])
            return x, y

        # Constrained dataset:
        x = np.zeros((n_points, self.input_shape))
        bounds = []
        for i, b in enumerate(self.input_bounds):
            if self.input_type[i] == "int":
                bounds.append((int(b[0]), int(b[1])))
            else:
                bounds.append((float(b[0]), float(b[1])))

        n_points_boundaries = n_points // 2
        for p in range(n_points_boundaries):
            cplex = cpx.Model()
            xvars = []
            for i,b in enumerate(self.input_bounds):
                if self.input_type[i] == "int":
                    xvars.append(cplex.integer_var(lb=b[0], ub=b[1], name="x"+str(i)))
                else:
                    xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))
            
            csts = self.constraint_cb(cplex, xvars)
            for pc in csts:
                cplex.add_constraint(*pc)

            # linear random objective (boundaries)
            obj = 0
            for var in xvars:
                obj += var * (np.random.uniform()*2-1)
            cplex.set_objective("max", obj)
            
            # solve
            cplex.set_time_limit(30)
            sol = cplex.solve()
            for i in range(self.input_shape):
                x[p, i] = sol["x"+str(i)]

        # Interpolation  
        for p in range(n_points_boundaries, n_points):
            pidx = np.random.choice(n_points_boundaries, 2, replace=False)
            p0, p1 = x[pidx[0]], x[pidx[1]]
            step = np.random.uniform()
            df = pd.DataFrame([p0, [np.nan]*self.input_shape, p1], 
                              index=[0, step, 1])
            df = df.interpolate(method="index")
            x[p] = df.iloc[1].values

        # eval fun
        y = np.zeros((n_points))
        for i in range(n_points):
            y[i] = self.fun(x[i, :])

        return x,y

    def get_grid(self, n_points):
        x_list = []
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x_list.append(np.arange(lb, ub, max(1, (ub-lb)//n_points)))
            else:
                x_list.append(np.arange(lb, ub, (ub-lb)/n_points))
        x = np.array(np.meshgrid(*x_list)).reshape(self.input_shape,-1).T
        y = np.zeros((x.shape[0]))
        for i in range(x.shape[0]):
            y[i] = self.fun(x[i, :])
        return x, y

##TFP utils

In [ ]:
#@title Build and plot

from matplotlib import cm
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

def build_probabilistic_regressor(input_shape, depth=4, width=20):
    mdl = tf.keras.Sequential()
    mdl.add(tf.keras.layers.Input(shape=(input_shape,), dtype='float32'))
    for i in range(depth):
        mdl.add(tf.keras.layers.Dense(width, activation='relu'))
    mdl.add(tf.keras.layers.Dense(2, activation='linear'))
    lf = lambda t: tfp.distributions.Normal(loc=t[:, :1], 
                                            scale=tf.keras.backend.exp(t[:, 1:]))
    mdl.add(tfp.layers.DistributionLambda(lf))
    return mdl

def dlambda_likelihood(y_true, dist):
    return -dist.log_prob(y_true)

def plot_prob_predictions(mdl, x, y):
    prob_pred = mdl(np.expand_dims(x, axis=1))
    pred = prob_pred.mean().numpy().ravel()
    std_pred = prob_pred.stddev().numpy().ravel()
    plt.plot(x, y, c="grey")
    plt.plot(x, pred)
    plt.fill_between(x, pred-std_pred, pred+std_pred, 
                    alpha=0.3, color='tab:blue', label='+/- std')

## Bayesian optimization

In [ ]:
#@title GPy opt - LCB

import GPy
from GPyOpt.methods import BayesianOptimization

def bayesian_opt_gpy(problem, iterations, starting_points):
    set_seed()

    x,y = problem.get_dataset(starting_points)
    if problem.input_shape == 1:
        x = x.reshape(-1, 1)

    bds = []
    for i,b in enumerate(problem.input_bounds):
        bds.append({
            'name': 'X'+str(i), 
            'type': 'continuous', 
            'domain': problem.input_bounds[i]})
        
    def wrapper(in_x):
        return problem.fun(in_x.ravel())

    kernel = GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=1.0)
    optimizer = BayesianOptimization(f=wrapper, 
                                    domain=bds,
                                    model_type='GP',
                                    kernel=kernel,
                                    acquisition_type ='EI',
                                    acquisition_jitter = 0.01,
                                    X=x,
                                    Y=y.reshape(-1,1),
                                    exact_feval=False,
                                    normalize_Y=True,
                                    maximize=False)

    optimizer.run_optimization(max_iter=iterations)
    optimizer.plot_acquisition()
    print("Min found:", optimizer.x_opt, optimizer.fx_opt)
    if ENABLE_WANDB:
        wandb.log({
            "min_found": np.min(optimizer.get_evaluations()[1]),
            "n_iterations": np.argmin(optimizer.get_evaluations()[1])+1-starting_points
        })
    return optimizer

## EML utils

In [ ]:
#@title Parse TFP model

from eml.net.reader import keras_reader

def parse_tfp(model):
    in_shape = model.input_shape[1]
    mdl_no_dist = tf.keras.Sequential()
    mdl_no_dist.add(tf.keras.layers.Input(shape=(in_shape,), dtype='float32'))
    for i in range(len(model.layers)-2):
        w = model.layers[i].weights[1].shape[0]
        mdl_no_dist.add(tf.keras.layers.Dense(w, activation='relu'))
    mdl_no_dist.add(tf.keras.layers.Dense(2, activation='linear'))

    mdl_no_dist.set_weights(model.get_weights())

    nn = keras_reader.read_keras_sequential(mdl_no_dist)
    return nn

In [ ]:
#@title Bounds

from eml.net.process import ibr_bounds, fwd_bound_tighthening

def propagate_bound(bkd, parsed_model, bounds):
    bounds = np.array(bounds)
    parsed_model.layer(0).update_lb(bounds[:,0])
    parsed_model.layer(0).update_ub(bounds[:,1])
    fwd_bound_tighthening(bkd, parsed_model, timelimit=30)
    print("Model output bounds:\n",parsed_model)
    return parsed_model

In [ ]:
#@title Encode model

from eml.backend import cplex_backend
from eml.net import embed

def embed_model(bkd, cplex, parsed_model, vtype, bounds):
    mean_lb = parsed_model.layer(-1).lb()[0]  # bounds computed with propagate bounds method
    mean_ub = parsed_model.layer(-1).ub()[0]
    std_lb = parsed_model.layer(-1).lb()[1]
    std_ub = parsed_model.layer(-1).ub()[1]

    xvars = []
    for i,b in enumerate(bounds):
        xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))

    yvars = [cplex.continuous_var(lb=mean_lb, ub=mean_ub, name="out_mean"), 
            cplex.continuous_var(lb=std_lb, ub=std_ub, name="out_std")]

    embed.encode(bkd, parsed_model, cplex, xvars, yvars, 'nn')
    return xvars, yvars

In [ ]:
#@title PWL helper

from eml.util import encode_pwl
from scipy.stats import norm


def pwl_exp(bkd, cplex, var, nnodes=7):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(math.exp, xx)))
    v = [cplex.continuous_var(lb=0, ub=np.max(yy), name="exp_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_abs(bkd, cplex, var):
    xx = np.array([-1, 0, 1])
    yy = np.array([1, 0, 1])
    v = [cplex.continuous_var(lb=0, ub=1), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_cdf(bkd, cplex, var, nnodes=11):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.cdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="ncdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_pdf(bkd, cplex, var, nnodes=11):
    xx = np.linspace(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.pdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="npdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_sample_dist(bkd, cplex, vars, samples, vtype, bounds, nnodes=20):
    mapf = lambda x: np.min(np.sum(np.abs(samples - x), axis=1)) / len(bounds)
    p = Problem(None, mapf, vtype, bounds)
    x,y = p.get_grid(nnodes)
    ub = np.diff(np.array(bounds), axis=1)
    ub = np.sum(ub)
    v = [cplex.continuous_var(lb=0, ub=ub, name="dist_out")]+vars
    encode_pwl(bkd, cplex, v, [y,*x.T])
    return v[0]

## 🔄 Optimization loop

In [ ]:
#@title Base Experiment

import time
import tensorflow_addons as tfa

class BaseExperiment:
    def __init__(self, problem, starting_points, iterations, 
                 epochs, lr, weight_decay, depth, width, batch_size, 
                 solver_timeout):
        set_seed()
        self.problem = problem
        self.starting_points = starting_points
        self.iterations = iterations
        self.epochs = epochs
        self.lr = lr
        self.weight_decay = weight_decay
        self.depth = depth
        self.width = width
        self.batch_size = batch_size
        self.solver_timeout = solver_timeout
        self.x_samples, self.y_samples = problem.get_dataset(starting_points)

    def train(self, keras_mdl):
        raise NotImplementedError

    def solver_optimization(self, keras_mdl):
        raise NotImplementedError

    def solution_log(self, solution):
        raise NotImplementedError

    def normalize_X(self, x):
        bounds = np.array(self.problem.input_bounds)
        return (x-bounds[:,0]) / (bounds[:,1]-bounds[:,0])

    def normalize_Y(self, y):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        return (y-min) / (max-min)

    def reverse_normalize_X(self, norm_x, milp_expr=False):
        bounds = np.array(self.problem.input_bounds)
        if milp_expr:  
            # for cplex variables, np array not supported
            bounds_range = np.squeeze(np.diff(bounds, axis=1))
            xvars = []
            for i,x in enumerate(norm_x):
                xvars.append(x * bounds_range[i] + bounds[i,0])
            return xvars
        return (bounds[:,1]-bounds[:,0])*norm_x + bounds[:,0]

    def reverse_normalize_Y(self, norm_y, stddev=False):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        if stddev:
            return norm_y*(max-min)
        return (max-min)*norm_y + min

    def plot(self, hstory, keras_mdl):
        if self.problem.input_shape <= 2:
            x,y = self.problem.get_grid(100)
            prob_pred = keras_mdl(self.normalize_X(x))
            pred = prob_pred.mean().numpy().ravel()
            pred = self.reverse_normalize_Y(pred)
            std_pred = prob_pred.stddev().numpy().ravel()
            std_pred = self.reverse_normalize_Y(std_pred, stddev=True)

        plt.plot(hstory.history["loss"])
        plt.savefig('train_loss.png')
        plt.show()
        
        fig = plt.figure(figsize=(15,10))
        if self.problem.input_shape == 1:   # 1D domain
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, 
                            alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(self.x_samples, self.y_samples, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples"])
            plt.savefig('chart.png')
            plt.show()
        elif self.problem.input_shape == 2:   # 2D domain
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(self.x_samples[:,0], self.x_samples[:,1], self.y_samples, color="orange")
            ax.scatter(x[:,0], x[:,1], y, alpha=0.15, color="lightgrey")
            ax.scatter(x[:,0], x[:,1], pred, alpha=0.3)
            ax.scatter(x[:,0], x[:,1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:,0], x[:,1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "GT", "predicted mean", "predicted CI"])
            plt.savefig('chart.png')
            plt.show()
        else:
            print("Plot not available for high dimensional domains.")
            print(f"X:\n{self.x_samples}\nY:\n{self.y_samples}")
            prob_pred = keras_mdl(self.normalize_X(self.x_samples))
            mu = self.reverse_normalize_Y(prob_pred.mean().numpy().ravel())
            print("mu pred\n",mu)
            print("diff\n",self.y_samples-mu)

    def run(self):
        not_improve = 0
        for iteration in range(self.iterations):
            # build NN
            print(f"Iteration {iteration}:", "="*20)
            optimizer = tfa.optimizers.AdamW(weight_decay=self.weight_decay,
                                             learning_rate=self.lr)
            keras_mdl = build_probabilistic_regressor(
                self.problem.input_shape, self.depth, self.width)
            keras_mdl.compile(optimizer=optimizer, loss=dlambda_likelihood)
            keras_mdl.summary()

            # train
            train_time = time.time()
            keras_mdl, hstory = self.train(keras_mdl)
            train_time = time.time() - train_time

            # plot loss and predictions if domain dim <= 2
            self.plot(hstory, keras_mdl)

            # MILP solver
            solver_time = time.time()
            sol = self.solver_optimization(iteration, not_improve, keras_mdl)
            solver_time = time.time() - solver_time

            if sol is None:
                print(f'Not feasible')
                break
            sol.solve_details.print_information()

            # retrieve solution
            opt_x = np.zeros(self.problem.input_shape)
            for i in range(self.problem.input_shape):
                opt_x[i] = sol["x"+str(i)]
            opt_x = self.reverse_normalize_X(opt_x)
            
            # retrieve integer solutions
            for i in range(self.problem.input_shape):
                if self.problem.input_type[i] == "int":
                    opt_x[i] = sol["int_x"+str(i)]

            # check if repeated data point and query obj function
            dists = np.sum(np.abs(self.x_samples - np.expand_dims(opt_x, 0)), axis=1)
            dmin = np.min(dists)
            dargmin = np.argmin(dists)
            # For non stocasthic problems, avoid resampling the same points, but giving much sample weight
            if not self.problem.stocasthic and dmin <= 1e-6: 
                print("Preventing query for an already known point !")
                opt_y = self.y_samples[dargmin]
            else:
                # BBF query
                opt_y = self.problem.fun(opt_x)

            # log new point
            emean = self.reverse_normalize_Y(sol['out_mean'])
            print(f"opt input={opt_x}, expected mean={emean}, gt={opt_y}")
            print(f"train time: {train_time}, solver time: {solver_time}")

            # not improve counter
            if opt_y < np.min(self.y_samples):
                not_improve = 0
            else:
                not_improve += 1
            
            # add new point to the dataset
            self.x_samples = np.concatenate((self.x_samples, np.expand_dims(opt_x, 0)))
            self.y_samples = np.append(self.y_samples, opt_y)

            # log and WandB
            if iteration == 0 and ENABLE_WANDB: # log starting points before the first iteration
                for j in range(self.starting_points):
                    wandb.log({"x_sample": self.x_samples[j], "y_sample": self.y_samples[j]})
                
            self.solution_log(sol)

            if ENABLE_WANDB: 
                if self.problem.input_shape <= 2:
                    wandb.log({"train_predictions": wandb.Image('chart.png')}, commit=False)
                wandb.log({
                    "train_loss": wandb.Image('train_loss.png'),
                    "x_sample": opt_x, "y_sample": opt_y, "y_min": np.min(self.y_samples),
                    "train_time": train_time, "solver_time": solver_time
                })
        # == end opt loop
        print(f"Min found: {np.min(self.y_samples)} in {np.argmin(self.y_samples)+1-self.starting_points} iterations")
        if ENABLE_WANDB:
            wandb.log({
                "min_found": np.min(self.y_samples),
                "n_iterations": np.argmin(self.y_samples)+1-self.starting_points
            })

        # Dump points for future studies
        dump_points = list(zip(self.x_samples, self.y_samples))
        with open('points.pkl', 'wb') as f:
            pickle.dump(dump_points, f)
        if ENABLE_WANDB:
            artifact = wandb.Artifact('datapoints', type='points')
            artifact.add_file("points.pkl")
            wandb.log_artifact(artifact)

        ymin = np.min(self.y_samples)
        yargmin = np.argmin(self.y_samples)
        xmin = self.x_samples[yargmin]
        return ymin, xmin

In [ ]:
#@title Base UCB

class BaseUCBExperiment(BaseExperiment):

    def __init__(self, beta_ucb, *args, **kwargs):
        super(BaseUCBExperiment, self).__init__(*args, **kwargs)
        self.beta = beta_ucb

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(cplex, rev_norm_xvars)
        
        ucb = -yvars[0] + self.beta * stddev
        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()

        print(cplex.solve_details)
        cplex.print_information()
        return sol

    def solution_log(self, solution):
        print(f"UCB: {solution.objective_value}")
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution['ucb'],
                "exp_err": solution['exp_out'] - math.exp(solution['out_std'])
            }, commit=False)

In [ ]:
#@title EarlyStop

class EarlyStop(BaseUCBExperiment):

    def __init__(self, patience, eval_points, *args, **kwargs):
        super(EarlyStop, self).__init__(*args, **kwargs)
        self.eval_points = eval_points
        self.x_val, self.y_val = self.problem.get_dataset(self.eval_points)
        self.x_val = self.normalize_X(self.x_val)
        self.y_val = self.normalize_Y(self.y_val)
        self.cb = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                    patience=patience, restore_best_weights=True)]
        
    def train(self, keras_mdl):
        print("Early stop training...")
        bs = self.batch_size if self.batch_size else self.x_samples.shape[0]

        norm_x = self.normalize_X(self.x_samples)
        norm_y = self.normalize_Y(self.y_samples)

        hstory = keras_mdl.fit(norm_x, norm_y, 
                               validation_data=(self.x_val, self.y_val),
                               batch_size=bs, epochs=self.epochs, 
                               verbose=0, callbacks=self.cb)

        return keras_mdl, hstory

In [ ]:
#@title Stop CI

class StopCICB(tf.keras.callbacks.Callback):

    def __init__(self, threshold, *args, **kwargs):
        super(StopCICB, self).__init__(*args, **kwargs)
        self.threshold = threshold
        self.x = None

    def on_epoch_end(self, epoch, logs={}):
        prob_pred = self.model(self.x)
        std_pred = prob_pred.stddev().numpy().ravel()
        if epoch % 100 == 0:
            print(epoch, "mean stddev", np.mean(std_pred))
        if np.mean(std_pred) < self.threshold:
            print("break condition on epoch", epoch, "with mean stddev", np.mean(std_pred))
            self.model.stop_training = True

class StopCI(BaseUCBExperiment):

    def __init__(self, ci_threshold, *args, **kwargs):
        super(StopCI, self).__init__(*args, **kwargs)
        self.ci_threshold = ci_threshold
        stop_ci = StopCICB(self.ci_threshold)
        self.cb = [stop_ci]

    def train(self, keras_mdl):
        print("Stop CI training...")
        bs = self.batch_size if self.batch_size else self.x_samples.shape[0]

        norm_x = self.normalize_X(self.x_samples)
        norm_y = self.normalize_Y(self.y_samples)

        self.cb[0].x = norm_x

        hstory = keras_mdl.fit(norm_x, norm_y,
                batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb)

        return keras_mdl, hstory

In [ ]:
#@title AugmentUniform

class AugmentUniform(StopCI):

    def __init__(self, num_aug, *args, **kwargs):
        super(AugmentUniform, self).__init__(*args, **kwargs)
        self.num_aug = num_aug
        self.x_aug_samples = None
        self.y_aug_samples = None

    def plot(self, hstory, keras_mdl):
        if self.problem.input_shape <= 2:
            x,y = self.problem.get_grid(100)
            prob_pred = keras_mdl(self.normalize_X(x))
            pred = prob_pred.mean().numpy().ravel()
            pred = self.reverse_normalize_Y(pred)
            std_pred = prob_pred.stddev().numpy().ravel()
            std_pred = self.reverse_normalize_Y(std_pred, stddev=True)

        plt.plot(hstory.history["loss"])
        plt.savefig('train_loss.png')
        plt.show()

        fig = plt.figure(figsize=(15,10))
        if self.problem.input_shape == 1:   # 1D domain
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, 
                            alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(self.x_aug_samples, self.y_aug_samples, c="grey")
            plt.scatter(self.x_samples, self.y_samples, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples", "augmented samples"])
            plt.savefig('chart.png')
            plt.show()
        elif self.problem.input_shape == 2:    # 2D domain
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(x[:,0], x[:,1], y, alpha=0.15, color="lightgrey")
            ax.scatter(self.x_aug_samples[:,0], self.x_aug_samples[:,1], self.y_aug_samples, c="grey", alpha=0.3)
            ax.scatter(self.x_samples[:,0], self.x_samples[:,1], self.y_samples, color="orange", alpha=1)
            ax.scatter(x[:,0], x[:,1], pred)
            ax.scatter(x[:,0], x[:,1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:,0], x[:,1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "augmented samples", "predicted mean", "predicted CI"])
            plt.savefig('chart.png')
            plt.show()
        else:
            print("Plot not available for high dimensional domains.")
            print(f"X:\n{self.x_samples}\nY:\n{self.y_samples}")

    def train(self, keras_mdl):
        print("Uniform augmented training...")
        self.x_aug_samples = np.concatenate((self.x_samples, 
                                        self.problem.get_dataset(self.num_aug)[0]))
        y_range = np.max(np.abs(self.y_samples))*2                              
        self.y_aug_samples = np.concatenate((self.y_samples, 
                                        (np.random.rand(self.num_aug)-0.5)*y_range))  

        sw = np.ones_like(self.x_aug_samples)  
        sw[:self.x_samples.shape[0]] = self.num_aug*2

        bs = self.batch_size if self.batch_size else self.x_aug_samples.shape[0]

        norm_x = self.normalize_X(self.x_aug_samples)
        norm_y = self.normalize_Y(self.y_aug_samples)
        self.cb[0].x = norm_x

        hstory = keras_mdl.fit(norm_x, norm_y,
            batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb, sample_weight=sw)
        
        return keras_mdl, hstory

In [ ]:
#@title LHS

class LHS(AugmentUniform):

    def train(self, keras_mdl):
        if self.problem.constraint_cb is not None:
            raise Error("Contrained objective not supported by augmented LHS.")
            
        print("LHS augmented training")
        n_datapoint = self.num_aug ** self.problem.input_shape

        aug_x = self.problem.get_grid(self.num_aug)[0]
        aug_x = np.concatenate((aug_x, aug_x))

        y_range = np.max(np.abs(self.y_samples))*2                              
        aug_y = np.concatenate(((np.random.rand(n_datapoint)-0.5)*y_range,
                                (np.random.rand(n_datapoint)-0.5)*y_range))
        self.x_aug_samples = np.concatenate((self.x_samples, aug_x))
        self.y_aug_samples = np.concatenate((self.y_samples, aug_y))

        sw = np.ones_like(self.x_aug_samples)  
        sw[:self.x_samples.shape[0]] = n_datapoint

        bs = self.batch_size if self.batch_size else self.x_aug_samples.shape[0]

        norm_x = self.normalize_X(self.x_aug_samples)
        norm_y = self.normalize_Y(self.y_aug_samples)
        self.cb[0].x = norm_x

        hstory = keras_mdl.fit(norm_x, norm_y,
            batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb, sample_weight=sw)
        
        return keras_mdl, hstory

In [ ]:
#@title MILP dist

class PWLMILPDist(StopCI):

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("Distance based UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        norm_x = self.normalize_X(self.x_samples)
        dist = pwl_sample_dist(bkd, cplex, xvars, norm_x, 
                               self.problem.input_type,
                               [[0,1]]*self.problem.input_shape, 
                               nnodes = min(20, self.x_samples.shape[0]))

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(cplex, rev_norm_xvars)

        ucb = -yvars[0] + self.beta * (stddev + dist)

        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()

        cplex.print_information()
        return sol

In [ ]:
#@title Hybrid

class Hybrid(PWLMILPDist, AugmentUniform):
    pass

In [ ]:
#@title FastMILP dist

class FastMILPDist(StopCI):

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("Distance based UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(bkd, parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        # Distance
        norm_x = self.normalize_X(self.x_samples)
        dist = cplex.continuous_var(lb=0, ub=1, name="dist")
        for row in range(norm_x.shape[0]):
            sd = 0
            for feature in range(norm_x.shape[1]):
                #sd += (norm_x[row, feature] - xvars[feature]) * (norm_x[row, feature] - xvars[feature]) # NOTE: non-convex
                sd += cplex.abs(norm_x[row, feature] - xvars[feature])
            cplex.add_constraint(sd >= dist*self.problem.input_shape)  # scale down the l1 dist with dimensions

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(cplex, rev_norm_xvars)
           
        # UCB
        ucb = -yvars[0] + self.beta * (stddev + dist)

        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()

        cplex.print_information()

        box = sys.stdout
        sys.stdout = open('cplex_model.txt', 'w')
        cplex.prettyprint()
        sys.stdout.close()
        sys.stdout = box

        return sol

    def solution_log(self, solution):
        print("Normalized Dist:", solution['dist'])
        print(f"UCB: {solution.objective_value}")
        print("mean:", solution['out_mean'])
        print("logstd:", solution['out_std'])
        print("truestd:", math.exp(solution['out_std']))
        print("stddev:", solution['exp_out'])
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution.objective_value,
                "norm_dist": solution['dist'],
                "exp_err": solution['exp_out'] - math.exp(solution['out_std'])
            }, commit=False)

In [ ]:
#@title Beta decay

class BetaDecay(StopCI):

    def __init__(self, lns_fixed, *args, **kwargs):
        super(BetaDecay, self).__init__(*args, **kwargs)
        self.lns_fixed = lns_fixed

    def solver_optimization(self, iteration, not_improve, keras_mdl):
        print("Distance based UCB solver...")

        ## K-Lipschitz with L1 dist
        k_lip = 0
        qlist = []
        for i in range(self.x_samples.shape[0]):
            for j in range(i+1, self.x_samples.shape[0]):
                delta_y = np.abs(self.y_samples[i]-self.y_samples[j])
                delta_x = np.sum(np.abs(self.x_samples[i]-self.x_samples[j]))/self.problem.input_shape
                if delta_x >= 1e-6: #eps to avoid inf
                    k_lip = max(k_lip, delta_y/delta_x)
                    qlist.append(delta_y/delta_x)
        print("K-Lipschitz max:", k_lip, 
              "95 percentile:", np.percentile(qlist, 95, interpolation='linear'))
        
        solution = None
        feature_fixed = self.problem.input_shape - (self.problem.input_shape // self.lns_fixed)
        for lns_i in range(self.lns_fixed):
            print("@"*20, "LSN iteration:", lns_i)
            cplex = cpx.Model()
            bkd = cplex_backend.CplexBackend()
            norm_x = self.normalize_X(self.x_samples)

            ## Beta
            if self.beta is not None:
                self.current_beta = self.beta
            else:
                self.current_beta = k_lip * ((1-iteration/ITERATIONS)**4)

            ## LNS
            best = norm_x[np.argmin(self.y_samples)]
            selection = np.random.choice(self.problem.input_shape, feature_fixed, replace=False)
            print("LNS Random selection", selection, "best:", best)
            pbounds = [[0,1]]*self.problem.input_shape
            for v in selection:
                pbounds[v] = [best[v], best[v]]
            
            ## NN encoding
            parsed_mdl = parse_tfp(keras_mdl)
            print("Propagating bounds...")
            propagate_bound(bkd, parsed_mdl, pbounds)
            xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                    self.problem.input_type,
                                    pbounds)

            # Handle integer variables
            for idx, bound in enumerate(self.problem.input_bounds):
                if self.problem.input_type[idx] == "int":
                    int_cst = cplex.integer_var(lb=bound[0], ub=bound[1], name="int_x"+str(idx))
                    cplex.add_constraint(int_cst == xvars[idx]*(bound[1]-bound[0])+bound[0])
            
            ## EXP PWL
            stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

            #################### Distance section
            # Sample distance
            sample_distance_list = []
            for row in range(norm_x.shape[0]):
                current_sample_dist = 0
                for feature in range(norm_x.shape[1]):
                    current_sample_dist += cplex.abs(norm_x[row, feature] - xvars[feature])
                sample_distance_list.append(current_sample_dist)
            
            ## triangular inequality
            def l1_dist(a,b): 
                return np.sum(np.abs(a-b))
            for id_xy in range(len(sample_distance_list)):
                s_xy = sample_distance_list[id_xy]
                for id_xz in range(id_xy, len(sample_distance_list)):
                    s_xz = sample_distance_list[id_xz]
                    cplex.add_constraint(s_xy <= s_xz + l1_dist(norm_x[id_xy],norm_x[id_xz]), "traingular inequality")

            # Min distance
            min_dist = cplex.continuous_var(lb=0, ub=self.problem.input_shape, name="dist")
            for current_sample_dist in sample_distance_list:
                cplex.add_constraint(current_sample_dist >= min_dist, "min dist")  # scale down the l1 dist with dimensions
            #################### END Distance section
            
            # Problem contraints
            if self.problem.constraint_cb is not None:
                rev_norm_xvars = self.reverse_normalize_X(xvars, True)
                csts = self.problem.constraint_cb(cplex, rev_norm_xvars)
                for pc in csts:
                    cplex.add_constraint(*pc)

            # UCB
            ucb = -yvars[0] + \
                self.current_beta * stddev + \
                (self.current_beta/self.problem.input_shape) * min_dist

            cplex.set_objective('max', ucb)
            cplex.set_time_limit(self.solver_timeout)

            cplex.parameters.mip.strategy.nodeselect = 2

            # CPLEX log
            cplex.context.solver.verbose = 5
            cplex.context.solver.log_output = True

            cplex.print_information()
            box = sys.stdout
            sys.stdout = open('cplex_model.txt', 'w')
            cplex.prettyprint()
            sys.stdout.close()
            sys.stdout = box

            print("Solving...")
            sol = cplex.solve()

            if sol is not None:
                if solution is not None:
                    if sol.objective_value > solution.objective_value:
                        solution = sol
                else:
                    solution = sol

        return solution

    def solution_log(self, solution):
        print("Beta:", self.current_beta)
        print("Normalized Dist:", solution['dist'])
        print(f"UCB: {solution.objective_value}")
        print("mean:", solution['out_mean'])
        print("logstd:", solution['out_std'])
        print("truestd:", math.exp(solution['out_std']))
        print("stddev:", solution['exp_out'])
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution.objective_value,
                "norm_dist": solution['dist'],
                "mean": solution['out_mean'],
                "stddev": solution['exp_out'],
                "exp_err": solution['exp_out'] - math.exp(solution['out_std']),
                "beta_ucb": self.current_beta
            }, commit=False)

## TPC

In [ ]:
#@title Quantization definitions

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.quantization.keras.default_8bit import default_8bit_quantize_scheme, default_8bit_quantize_registry, default_8bit_quantizers
from tensorflow_model_optimization.quantization.keras.quantizers import LastValueQuantizer, MovingAverageQuantizer
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer as ql
from tensorflow_model_optimization.quantization.keras import quantize_annotate_model, QuantizeConfig


class ConvWeightsQuantizer(LastValueQuantizer):
  """Quantizer for handling weights in Conv2D/DepthwiseConv2D layers."""
  def __init__(self, bits):
        super(ConvWeightsQuantizer, self).__init__(
            num_bits=bits, per_axis=True, symmetric=True, narrow_range=True)
  def build(self, tensor_shape, name, layer):
    min_weight = layer.add_weight(
        name + '_min',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(-6.0),
        trainable=False)
    max_weight = layer.add_weight(
        name + '_max',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(6.0),
        trainable=False)
    return {'min_var': min_weight, 'max_var': max_weight}

class QConf(default_8bit_quantize_registry.Default8BitQuantizeConfig):
    def __init__(self, bits, conv, *args, **kwargs):
        super(QConf, self).__init__(*args, **kwargs)
        self.bits = bits
        if conv:
            self.weight_quantizer = ConvWeightsQuantizer(bits)
        else:
            self.weight_quantizer = LastValueQuantizer(
                num_bits=self.bits, per_axis=False, symmetric=True, narrow_range=True)
        self.activation_quantizer = MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)

class QAct(default_8bit_quantize_registry.Default8BitActivationQuantizeConfig):
    def __init__(self, bits, *args, **kwargs):
        super(QAct, self).__init__(*args, **kwargs)
        self.bits = bits

    def get_output_quantizers(self, layer):
        self._assert_activation_layer(layer)
        if not hasattr(layer.activation, '__name__'):
            raise ValueError('Activation {} not supported by '
                            'Default8BitActivationQuantizeConfig.'.format(
                                layer.activation))
        if layer.activation.__name__ in ['relu', 'swish']:
            return [MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)]
        elif layer.activation.__name__ in ['linear', 'softmax', 'sigmoid', 'tanh']:
            return []
        raise ValueError('Activation {} not supported by '
                        'Default8BitActivationQuantizeConfig.'.format(
                            layer.activation))

class QReg(default_8bit_quantize_registry.Default8BitQuantizeRegistry):
    def __init__(self, bitlist, *args, **kwargs):
        super(QReg, self).__init__(*args, **kwargs)
        self.bitlist = bitlist
        self.counter = -1

    def get_quantize_config(self, layer):
        self.counter += 1
        quantize_info = self._get_quantize_info(layer.__class__)
        if layer.name.startswith('activation'):
            return QAct(self.bitlist[self.counter])
        return QConf(self.bitlist[self.counter], 
                     layer.name.startswith('conv'),  # enable ConvWeightsQuantizer
                     quantize_info.weight_attrs,
                     quantize_info.activation_attrs,
                     quantize_info.quantize_output)

class QScheme(default_8bit_quantize_scheme.Default8BitQuantizeScheme):
    def __init__(self, bitlist, *args, **kwargs):
        super(QScheme, self).__init__(*args, **kwargs)
        self.bitlist = bitlist

    def get_quantize_registry(self):
        return QReg(self.bitlist)

# 🚀 Experiments

In [ ]:
#@title 🎯 Targets

# Functions
################################################################################

def fun_polynomial(x):
    return 0.5*(0.1*math.pow(5*x-1, 4) - 0.4*math.pow(5*x-1, 3) + 0.5*(5*x-1))

def fun_ackley(x):
    x1,x2 = x[0]-1,x[1]-2   # center in 1,2 to remove the 0 bias of nn
    return -20*math.exp(-0.2*math.sqrt(0.5*(x1*x1+x2*x2))) - \
        math.exp(0.5*(math.cos(2*math.pi*x1)+math.cos(2*math.pi*x2))) + \
        math.e + 20

def build_keane_bump(dim):
    def f(x):
        v1=0
        for i in range(dim):
            v1 += math.pow(math.cos(x[i]),4)
        v2=2
        for i in range(dim):
            v2 *= math.pow(math.cos(x[i]),2)
        v3=1e-6 #zero div
        for i in range(dim):
            v3 += (i+1)*x[i]*x[i]
        v3 = math.sqrt(v3)
        return -abs((v1-v2)/v3)

    vtypes = ["real"]*dim
    bounds = [[0, 10]]*dim
    return f, vtypes, bounds

def build_ackley(dim):
    def f(x):
        v1=0
        for i in range(dim):
            v1 += x[i]*x[i]
        v1 = math.exp(-0.2*math.sqrt(v1/dim))
        v2=0
        for i in range(dim):
            v2 += math.cos(2*math.pi*x[i])
        v2 = math.exp(v2/dim)
        return -20*v1-v2+20+math.e

    vtypes = ["real"]*dim
    bounds = [[-5, 10]]*dim
    return f, vtypes, bounds

def fun_mccormick(x):
    x1,x2 = x[0],x[1]
    return math.sin(x1+x2) + math.pow(x1-x2,2)-1.5*x1+2.5*x2+1

def build_rosenbrock(rosenbrock_dim):
    def f(x):
        y=0
        for i in range(rosenbrock_dim-1):
            y+= 100 * math.pow(x[i+1] - x[i]*x[i], 2) + math.pow(1-x[i], 2)
        return y
    vtypes = ["real"]*rosenbrock_dim
    bounds = [[-2, 2]]*rosenbrock_dim
    return f, vtypes, bounds

def build_tpc():
    cifar10 = tf.keras.datasets.cifar10
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
    dataset_mean = train_images.mean(axis=(0,1,2))
    dataset_std = train_images.std(axis=(0,1,2))
    train_images = (train_images - dataset_mean) / dataset_std
    test_images = (test_images - dataset_mean) / dataset_std

    !mkdir pretrained_resnet18
    !mkdir pretrained_resnet18/variables
    !wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/d9d4d8f866df84014e528bb3c5617816 -O  pretrained_resnet18/variables/variables.data-00000-of-00001
    !wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/4901af0e55327757ca7d7380b353279f -O  pretrained_resnet18/variables/variables.index
    !wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/0a1a30ebb8498c7adaab17365283b563 -O  pretrained_resnet18/keras_metadata.pb
    !wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/7c8a4682f521bac78f8a89b70342675b -O  pretrained_resnet18/saved_model.pb

    def tpc(x):
        print("Query for x=", x)
        model = tf.keras.models.load_model("pretrained_resnet18")
        model = quantize_annotate_model(model)
        q_aware_model = tfmot.quantization.keras.quantize_apply(model, QScheme(x))
        q_aware_model.compile(optimizer="adam",
                        loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        # q_aware_model.summary()

        q_aware_model.fit(train_images, train_labels,
                        batch_size=512, epochs=5, validation_split=0.1,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                            patience=5, restore_best_weights=True)])
        
        q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
            test_images, test_labels, verbose=0)
        
        return -q_aware_model_accuracy
    return tpc

# Constraints 
################################################################################

def cst_paper(cplex, xvars):
    r = 5
    acc1 = 0
    for xi in xvars:
        acc1 += xi*xi
    acc2 = 0
    for xi in xvars:
        acc2 += xi
    return [[acc1 <= r*r, "center_dist"], [acc2 <= 0, "cst2"]]

def cst_paper2(cplex, xvars):
    return [[sum(xvars) <= 7.5*len(xvars), 'sum']]

def cst_tpc_size(cplex, xvars):
    return [[sum(xvars) <= len(xvars)*2.5, 'cst_size']]

def cst_disk(cplex, xvars):     # x1^2 + x2^2 < r^2
    x1,x2 = xvars
    center = [1,2]
    r = 0.5
    x1 -= center[0]
    x2 -= center[1]
    cplex.add_constraint(x1*x1 + x2*x2 <= r*r)

def cst_sphere(cplex, xvars):     # x1^2 + x2^2 + x3^2 < r^2
    x1,x2,x3 = xvars
    center = [-1,0.5,0.6]
    r = 1.2
    x1 -= center[0]
    x2 -= center[1]
    x3 -= center[2]
    cplex.add_constraint(x1*x1 + x2*x2 + x3*x3 <= r*r)

def cst_leq(cplex, xvars):     # x1 < x2
    x1,x2 = xvars
    cplex.add_constraint(x1 <= x2)

# Problems
################################################################################

PROBLEM_POLYNOMIAL = Problem("polynomial", 
                             fun_polynomial, 
                             ["real"],
                             [[0, 1]])

PROBLEM_ACKLEY = Problem("ackley", 
                         fun_ackley, 
                         ["real", "real"],
                         [[-3, 3], [-3, 3]])

PROBLEM_CST_ACKLEY10D = Problem("constrained_ackley10D", 
                             *build_ackley(10),  
                             cst_paper)

PROBLEM_INT_ACKLEY = Problem("int_ackley", 
                             fun_ackley, 
                             ["int", "real"],
                             [[-3, 3], [-2, 2]],
                             cst_leq)

PROBLEM_MCCORMICK = Problem("mccormick", 
                            fun_mccormick, 
                            ["real", "real"],
                            [[-1.5, 4], [-3, 4]])

PROBLEM_ROSENBROCK2D = Problem("rosenbrock2D", build_rosenbrock(2)[0], 
                               ["real", "real"], [[-2, 2]]*2)

PROBLEM_ROSENBROCK40D = Problem("rosenbrock40D", *build_rosenbrock(40))

PROBLEM_KEANE2D = Problem("keane2D", *build_keane_bump(2))

PROBLEM_TPC = ProblemQRS("tpc", build_tpc(), ["int"]*41, [[2, 8]]*41, cst_tpc_size, stocasthic=True)

In [ ]:
#@title 📈 Parameters and WandB

ITERATIONS =                                  50#@param {type:"integer"}
STARTING_POINTS =                              5#@param {type:"integer"}

EPOCHS =                                    999#@param {type:"integer"}  
PATIENCE =                                  None  #@param {type:"raw"}

EVAL_POINTS = None                                   #@param {type:"raw"}
LR =                                        1e-3#@param {type:"number"}
WEIGHT_DECAY =                                        1e-4#@param {type:"number"}
BATCH_SIZE = None                               #@param {type:"raw"} None = all samples in 1 batch
DEPTH =                                        1#@param {type:"integer"}
WIDTH =                                       200#@param {type:"integer"}

BETA_UCB =                                     1#@param {type:"raw"}

AUG_POINTS =                                 None#@param {type:"raw"}

SOLVER_TIMEOUT =                              120#@param {type:"integer"}

CI_THRESHOLD =              0.05#@param {type:"raw"} 
LNS_FIXED =              1#@param {type:"integer"} 

# set if you plan to log on wandb
ENABLE_WANDB = True                            #@param {type:"boolean"}        

if ENABLE_WANDB and "wandb" not in sys.modules:
    !pip install wandb > /dev/null
    !wandb login
    import wandb

def init_wandb(experiment_name, run_id):
    if run_id is not None: 
        wandb.init(project='eml', id=run_id, resume='allow')
    else:
        wandb.init(project='eml', name=experiment_name)

        wandb.config.iterations = ITERATIONS
        wandb.config.starting_points = STARTING_POINTS
        wandb.config.epochs = EPOCHS
        wandb.config.patience = PATIENCE
        wandb.config.eval_points = EVAL_POINTS
        wandb.config.lr = LR
        wandb.config.weight_decay = WEIGHT_DECAY
        wandb.config.batch_size = BATCH_SIZE
        wandb.config.depth = DEPTH
        wandb.config.width = WIDTH
        wandb.config.beta_ucb = BETA_UCB
        wandb.config.aug_points = AUG_POINTS
        wandb.config.solver_timeout = SOLVER_TIMEOUT
        wandb.config.ci_threshold = CI_THRESHOLD
        wandb.config.lns_fixed = LNS_FIXED

def run_experiment(name, target):
    if name == "BayesianOptimization":
        optimizer = bayesian_opt_gpy(target, ITERATIONS, STARTING_POINTS)
        return optimizer
        
    if name == "EarlyStop":
        instance = EarlyStop(
            PATIENCE, EVAL_POINTS, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "StopCI":
        instance = StopCI(
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "AugmentUniform":
        instance = AugmentUniform(
            AUG_POINTS, 
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "AugmentLHS":
        instance = LHS(
            AUG_POINTS, 
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "PWLMILPDist":
        instance = PWLMILPDist(
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "Hybrid":
        instance = Hybrid(
            AUG_POINTS, 
            CI_THRESHOLD,
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance

    if name == "FastMILPDist":
        instance = FastMILPDist(
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
    
    if name == "BetaDecay":
        instance = BetaDecay(
            LNS_FIXED,
            CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance

    raise AttributeError("invalid method")

In [ ]:
#@title ▶️ Run experiment

# set if starting a new run
EXPERIMENT_NAME = "ackley10_vanilla"      #@param {type:"string"}
# set to None if starting a new run
RUN_ID = None                                   #@param {type:"raw"}

if ENABLE_WANDB:
    init_wandb(EXPERIMENT_NAME, RUN_ID)

INSTANCE = None
try:
    INSTANCE = run_experiment("BetaDecay", PROBLEM_CST_ACKLEY10D)
finally:
    if ENABLE_WANDB:
        wandb.finish()

In [ ]:
init_wandb("rosen2d", RUN_ID)
INSTANCE = None
try:
    INSTANCE = run_experiment("BetaDecay", PROBLEM_ROSENBROCK2D)
finally:
    if ENABLE_WANDB:
        wandb.finish()

init_wandb("kean2d", RUN_ID)
INSTANCE = None
try:
    INSTANCE = run_experiment("BetaDecay", PROBLEM_KEANE2D)
finally:
    if ENABLE_WANDB:
        wandb.finish()

# 🧪 Test

## Problem

In [ ]:
def test_problem():
    def f0(x):
        return 0.5*(0.1*math.pow(5*x-1, 4) - 0.4*math.pow(5*x-1, 3) + 0.5*(5*x-1))
    p0 = Problem(None, f0, [[0, 1]])

    def f1(x):
        x1,x2 = x[0],x[1]
        return 10 * math.sin(x1) * math.sin(x2) 
    p1 = Problem(None, f1, [[-1, 1], [-1, 1]])

    def ackley_fun(x):
        x1,x2 = x[0],x[1]
        return -20*math.exp(-0.2*math.sqrt(0.5*(x1*x1+x2*x2))) - \
            math.exp(0.5*(math.cos(2*math.pi*x1)+math.cos(2*math.pi*x2))) + \
            math.e + 20
    ackley_problem = Problem(None, ackley_fun, [[-3, 3], [-3, 3]])

    print("f0 grid")
    g0 = p0.get_grid(10)
    x0, y0 = g0[0], g0[1]
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(x0, y0)
    plt.show()

    print("f1 grid")
    g1 = p1.get_grid(10)
    x1, y1 = g1[0], g1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("ackley grid")
    g1 = ackley_problem.get_grid(100)
    x1, y1 = g1[0], g1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("f0 dataset")
    d0 = p0.get_dataset(10)
    x0, y0 = d0[0], d0[1]
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(x0, y0)
    plt.show()

    print("f1 dataset")
    d1 = p1.get_dataset(100)
    x1, y1 = d1[0], d1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("ackley dataset")
    d1 = ackley_problem.get_dataset(10000)
    x1, y1 = d1[0], d1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()
    
test_problem()

##TFP

In [ ]:
#@title Whole train set

def tfp_test1():
    set_seed()

    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=1, callbacks=cb)

    sorted = x_val.argsort()
    plot_prob_predictions(mdl_prob, x_val[sorted], y_val[sorted])

tfp_test1()

In [ ]:
#@title Few points linear

def tfp_test2():
    set_seed()

    x_samples = np.random.rand(5)
    y_samples = x_samples

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_samples.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_samples, y_samples,
            batch_size=bs, epochs=1000, verbose=1)

    sorted = x_samples.argsort()
    plot_prob_predictions(mdl_prob, x_samples[sorted], y_samples[sorted])

tfp_test2()

In [ ]:
#@title Different variance

def tfp_test3():
    set_seed()

    points = np.random.rand(50)
    noise = np.random.rand(20) / 5
    x_samples = points
    y_samples = np.copy(x_samples)
    sorted_idx = x_samples.argsort()
    y_samples[sorted_idx[:20]] += noise

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_samples.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_samples, y_samples,
            batch_size=bs, epochs=1000, verbose=1)

    sorted = x_samples.argsort()
    plot_prob_predictions(mdl_prob, x_samples[sorted], y_samples[sorted])

tfp_test3()

## EML

In [ ]:
#@title Minimize cost_fn

def eml_test1():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=0, callbacks=cb)
    
    #solve
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    cplex.set_objective('min', yvars[0])
    cplex.set_time_limit(30)

    sol = cplex.solve()

    print(f'feasible: {sol is not None}')
    print(cplex_backend.model_to_string(cplex))
    print(sol.display())

    opt_x = sol.get_value("x0")
    opt_mean = sol.get_value("out_mean")
    opt_std = math.pow(math.e, sol.get_value("out_std"))
    print(opt_x, opt_mean, opt_std)

    dist = mdl_prob(np.array([[opt_x]]))
    assert dist.mean() - opt_mean < 1e-6
    assert dist.stddev() - opt_std < 1e-6

eml_test1()

In [ ]:
#@title Test pwl

def eml_test2():
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    inp = cplex.continuous_var(lb=0, ub=1, name="test_in")
    out = pwl_normal_pdf(bkd, cplex, inp, nnodes=11)
    cplex.set_objective('max', out)
    cplex.set_time_limit(30)

    cplex.prettyprint()
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    print(sol)

    assert sol.get_value("test_in") == 0

eml_test2()

In [ ]:
#@title Minimize EI (non convex error)

#   imp = mu - opt_mean - 0.1
#   Z = imp / var
#   ei = imp * norm.cdf(Z) + var * norm.pdf(Z)

def eml_test3():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=0, callbacks=cb)
    
    #solve
    epsilon = 0.1
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl)
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl)

    # EI
    # find min mean (max improvement)
    cplex.set_objective('min', yvars[0])
    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("in")
        opt_mean = sol.get_value("out_mean")
        opt_std = math.pow(math.e, sol.get_value("out_std"))
        print(opt_x, opt_mean, opt_std)

    # compute ei
    epsilon = 0.1
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    imp = yvars[0] - opt_mean - epsilon
    stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=11)
    Z = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="Z")
    cplex.add_constraint(Z * stddev == imp) # Z = imp / exp(yvars[1])
    ncdf = pwl_normal_cdf(bkd, cplex, Z, nnodes=11)
    npdf = pwl_normal_pdf(bkd, cplex, Z, nnodes=11)
    ei = imp * ncdf + stddev * npdf
    cplex.set_objective('min', ei)

    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("x0")
    print(sol)

eml_test3()

In [ ]:
#@title Minimize UCB

def eml_test4():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=500, verbose=0, callbacks=cb)
    
    #solve
    beta = 0.01
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    ucb = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="ucb")
    stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=11)
    cplex.add_constraint(ucb == -yvars[0] + beta * stddev) # max -f = min f

    cplex.set_objective('max', ucb)

    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("x0")
        print(sol)
    return sol, cplex
eml_test4()

##TPC

In [ ]:
#@title ResNet18

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add, Input, ReLU, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model


def resnet_block(channels, down_sample=False):
    strides = [2, 1] if down_sample else [1, 1]
    KERNEL_SIZE = (3, 3)
    INIT_SCHEME = "he_normal"

    relu1 = ReLU()
    relu2 = ReLU()

    conv_1 = Conv2D(channels, strides=strides[0],
                       kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
    bn_1 = BatchNormalization()
    conv_2 = Conv2D(channels, strides=strides[1],
                       kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
    bn_2 = BatchNormalization()
    merge = Add()

    if down_sample:
        # perform down sampling using stride of 2, according to [1].
        res_conv = Conv2D(
            channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
        res_bn = BatchNormalization()

    def call(inputs):
        res = inputs
        x = conv_1(inputs)
        x = bn_1(x)
        x = Activation('relu')(x)
        x = conv_2(x)
        x = bn_2(x)

        if down_sample:
            res = res_conv(res)
            res = res_bn(res)
        # if not perform down sample, then add a shortcut directly
        x = merge([x, res])
        out = Activation('relu')(x)
        return out

    return call

def build_resnet():
    ks = 64
    inputs = Input(shape=(32, 32, 3))
    
    relu1 = ReLU()
    conv_1 = Conv2D(ks, (3, 3), strides=1,
                       padding="same", kernel_initializer="he_normal")
    init_bn = BatchNormalization()
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
    res_1_1 = resnet_block(ks)
    res_1_2 = resnet_block(ks)
    res_2_1 = resnet_block(ks*2, down_sample=True)
    res_2_2 = resnet_block(ks*2)
    res_3_1 = resnet_block(ks*4, down_sample=True)
    res_3_2 = resnet_block(ks*4)
    res_4_1 = resnet_block(ks*8, down_sample=True)
    res_4_2 = resnet_block(ks*8)

    avg_pool = GlobalAveragePooling2D()
    flat = Flatten()
    fc = Dense(10, activation="softmax")

    out = conv_1(inputs)
    out = init_bn(out)
    out = Activation('relu')(out)
    out = pool_2(out)
    for res_block in [res_1_1, res_1_2, res_2_1, res_2_2, res_3_1, res_3_2, res_4_1, res_4_2]:
        out = res_block(out)
    out = avg_pool(out)
    out = flat(out)
    out = fc(out)

    return Model(inputs, out)

In [ ]:
#@title CIFAR10 pretraining

set_seed()

# Load CIFAR10 dataset
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

dataset_mean = train_images.mean(axis=(0,1,2))
dataset_std = train_images.std(axis=(0,1,2))

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = (train_images - dataset_mean) / dataset_std
test_images = (test_images - dataset_mean) / dataset_std

# Build float model
model = build_resnet()
model.compile(optimizer="adam",
                loss='sparse_categorical_crossentropy', metrics=["accuracy"])
model.summary()

# Train float model
model.fit(
  train_images,
  train_labels,
  batch_size=128,
  epochs=50,
  callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
            patience=5, restore_best_weights=True)],
  validation_split=0.1,
)

In [ ]:
#@title Pretrained persistence: store

model.save("pretrained_resnet18")
artifact = wandb.Artifact('resnet18', type='model')
artifact.add_dir("pretrained_resnet18")
wandb.init(project='eml', name="artifact_resnet18")
wandb.log_artifact(artifact)
wandb.finish()

In [ ]:
#@title Pretrained persistence: load

! mkdir pretrained_resnet18
! mkdir pretrained_resnet18/variables

! wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/d9d4d8f866df84014e528bb3c5617816 -O  pretrained_resnet18/variables/variables.data-00000-of-00001
! wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/4901af0e55327757ca7d7380b353279f -O  pretrained_resnet18/variables/variables.index
! wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/0a1a30ebb8498c7adaab17365283b563 -O  pretrained_resnet18/keras_metadata.pb
! wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6NTU2NTg0NjE=/7c8a4682f521bac78f8a89b70342675b -O  pretrained_resnet18/saved_model.pb

model = tf.keras.models.load_model("pretrained_resnet18")

In [ ]:
#@title Quantization definitions

import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.quantization.keras.default_8bit import default_8bit_quantize_scheme, default_8bit_quantize_registry, default_8bit_quantizers
from tensorflow_model_optimization.quantization.keras.quantizers import LastValueQuantizer, MovingAverageQuantizer
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer as ql
from tensorflow_model_optimization.quantization.keras import quantize_annotate_model, QuantizeConfig


class ConvWeightsQuantizer(LastValueQuantizer):
  """Quantizer for handling weights in Conv2D/DepthwiseConv2D layers."""
  def __init__(self, bits):
        super(ConvWeightsQuantizer, self).__init__(
            num_bits=bits, per_axis=True, symmetric=True, narrow_range=True)
  def build(self, tensor_shape, name, layer):
    min_weight = layer.add_weight(
        name + '_min',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(-6.0),
        trainable=False)
    max_weight = layer.add_weight(
        name + '_max',
        shape=(tensor_shape[-1],),
        initializer=tf.keras.initializers.Constant(6.0),
        trainable=False)
    return {'min_var': min_weight, 'max_var': max_weight}

class QConf(default_8bit_quantize_registry.Default8BitQuantizeConfig):
    def __init__(self, bits, conv, *args, **kwargs):
        super(QConf, self).__init__(*args, **kwargs)
        self.bits = bits
        if conv:
            self.weight_quantizer = ConvWeightsQuantizer(bits)
        else:
            self.weight_quantizer = LastValueQuantizer(
                num_bits=self.bits, per_axis=False, symmetric=True, narrow_range=True)
        self.activation_quantizer = MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)

class QAct(default_8bit_quantize_registry.Default8BitActivationQuantizeConfig):
    def __init__(self, bits, *args, **kwargs):
        super(QAct, self).__init__(*args, **kwargs)
        self.bits = bits

    def get_output_quantizers(self, layer):
        self._assert_activation_layer(layer)
        if not hasattr(layer.activation, '__name__'):
            raise ValueError('Activation {} not supported by '
                            'Default8BitActivationQuantizeConfig.'.format(
                                layer.activation))
        if layer.activation.__name__ in ['relu', 'swish']:
            return [MovingAverageQuantizer(
            num_bits=self.bits, per_axis=False, symmetric=False, narrow_range=False)]
        elif layer.activation.__name__ in ['linear', 'softmax', 'sigmoid', 'tanh']:
            return []
        raise ValueError('Activation {} not supported by '
                        'Default8BitActivationQuantizeConfig.'.format(
                            layer.activation))

class QReg(default_8bit_quantize_registry.Default8BitQuantizeRegistry):
    def __init__(self, bitlist, *args, **kwargs):
        super(QReg, self).__init__(*args, **kwargs)
        self.bitlist = bitlist
        self.counter = -1

    def get_quantize_config(self, layer):
        self.counter += 1
        quantize_info = self._get_quantize_info(layer.__class__)
        if layer.name.startswith('activation'):
            return QAct(self.bitlist[self.counter])
        elif layer.name.startswith('conv'):
            return QConf(self.bitlist[self.counter], True,
                     quantize_info.weight_attrs,
                     quantize_info.activation_attrs,
                     quantize_info.quantize_output)
        return QConf(self.bitlist[self.counter], False,
                     quantize_info.weight_attrs,
                     quantize_info.activation_attrs,
                     quantize_info.quantize_output)

class QScheme(default_8bit_quantize_scheme.Default8BitQuantizeScheme):
    def __init__(self, bitlist, *args, **kwargs):
        super(QScheme, self).__init__(*args, **kwargs)
        self.bitlist = bitlist

    def get_quantize_registry(self):
        return QReg(self.bitlist)

In [ ]:
#@title QAT

bitlist = [3., 7., 2., 6., 2., 2., 5., 5., 6., 2., 2., 6., 2., 2., 5., 2., 4.,
       2., 4., 7., 7., 2., 2., 2., 7., 3., 3., 2., 5., 2., 2., 5., 2., 8.,
       5., 5., 2., 5., 6., 5., 7.]
model = quantize_annotate_model(model)
q_aware_model = tfmot.quantization.keras.quantize_apply(model, QScheme(bitlist))
q_aware_model.compile(optimizer="adam",
                loss='sparse_categorical_crossentropy', metrics=["accuracy"])
q_aware_model.summary()

q_aware_model.fit(train_images, train_labels,
                  batch_size=512, epochs=5, validation_split=0.1,
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                    patience=5, restore_best_weights=True)])

model.compile(optimizer="adam",
                loss='sparse_categorical_crossentropy', metrics=["accuracy"])
# Compare float vs quantized accuracy
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

# TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

In [ ]:
#@title TFLite vs TF model

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()
test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

In [ ]:
#@title Compare size

import tempfile

# # Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# # Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))